In [1]:
import sys, os
sys.path.append("../..")
sys.path.append("..")
sys.path.append(os.getcwd())

from matplotlib import pyplot as plt
from sklearn.metrics import r2_score
import numpy as np
import pandas as pd
import copy
import pickle

from mrsc.src.model.SVDmodel import SVDmodel
from mrsc.src.model.Target import Target
from mrsc.src.model.Donor import Donor
from mrsc.src.synthcontrol.mRSC import mRSC
from mrsc.src.importData import *
import mrsc.src.utils as utils

from itertools import combinations, product

def prepareData(stats):
    # transform stats to a dictionary composed of df's for each stat
    # the stats are re-calculated to get one stat for each year
    metricsPerGameColNames = ["PTS","AST","TOV","TRB","STL","BLK","3P"]
    metricsPerGameDict = getMetricsPerGameDict(stats, metricsPerGameColNames)

    metricsPerCentColNames = ["FG","FT"]
    metricsPerCentDict = getMetricsPerCentDict(stats, metricsPerCentColNames)

    metricsWeightedColNames = ["PER"]
    metricsWeightedDict = getMetricsWeightedDict(stats, metricsWeightedColNames)

    allMetricsDict = {**metricsPerGameDict, **metricsPerCentDict, **metricsWeightedDict}
    allPivotedTableDict = getPivotedTableDict(allMetricsDict)
    allMetrics = list(allMetricsDict.keys())
    return allPivotedTableDict, allMetrics

def getTopPlayers(stats, year, metric, n):
    stats = stats[stats.Year == year]
    stats = stats.groupby('Player').mean().reset_index()
    stats_sorted = stats[stats.Year == year].sort_values(metric, ascending = False).reset_index(drop=True)
    return stats_sorted[["Player"]][:n]

# bug in playerName
def getBenchmark(target, metrics_to_use, pred_interval):    
    target_data, nanIndex = target.concat(metrics_to_use)
    num_k = len(metrics_to_use)
    interv_index = int(target_data.shape[1]/num_k - pred_interval)
    total_index = int(interv_index + 1)
    
    # true
    true = utils.get_postint_data(target_data, interv_index, total_index, num_k).T
    true.index = metrics_to_use
    
    # predictions
    history = utils.get_preint_data(target_data, interv_index, total_index, num_k)
    pred = []
    for i in range(num_k):
        pred.append(history.iloc[:,i*interv_index:(i+1)*interv_index].mean(axis=1).to_list())

    #pred = pd.DataFrame(pred, index=metrics_to_use, columns = [playerName]) #bug
    pred = pd.DataFrame(pred, index=metrics_to_use, columns = [target.key])
    return true, pred

def getR2(true, pred, bench):
    ss_res = pd.DataFrame((true.values - pred.values)**2, index=true.index).sum(axis=1)
    ss_tot = pd.DataFrame((true.values - bench.values)**2, index=true.index).sum(axis=1)
    return (1-ss_res/ss_tot).to_frame(name = pred.columns.values[0])

In [2]:
def plot_pred(pred_traj, true_traj, markers_on, metric, playerName, dir_name):
    dir_name_metric = dir_name + metric + '/'
    plt.figure()
    plt.plot(pred_traj, marker='o', markevery=markers_on, color='blue', label='Prediction')
    plt.plot(true_traj, marker='o', color='red', label='True')
    plt.xticks(range(len(true_traj)), range(1, len(true_traj)+1))
    #plt.legend(loc='best')
    plt.legend(loc='upper left')
    plt.xlabel('Years in NBA')
    plt.ylabel(metric)
    plt.title(playerName + ': ' + metric)
    file_name = dir_name_metric + playerName + '.png'
    plt.savefig(file_name, bbox_inches='tight')
    plt.close()   
    
def annual_predictions(playerNames, allPivotedTableDict, donor, pred_interval, metrics, pred_metrics,
                      threshold, donorSetup, denoiseSetup, regression_method, verbose, dir_name, top_players):
    all_pred = pd.DataFrame()
    all_true = pd.DataFrame()
    all_bench = pd.DataFrame()
    all_R2 = pd.DataFrame()
    
    for playerName in playerNames:
        if playerName in top_players:
            dir_name_player = dir_name + 'top_players/'
        else:
            dir_name_player = dir_name + 'rest/'
                
        target = Target(playerName, allPivotedTableDict)
        mrsc = mRSC(donor, target, pred_interval, probObservation=1)        
        player_pred = pd.DataFrame()
        player_true = pd.DataFrame()
        
        # benchmark
        true, benchmark = getBenchmark(target, pred_metrics, pred_interval)
        
        for metric in metrics:
            mrsc.fit_threshold(metric, threshold, donorSetup, denoiseSetup,regression_method, verbose)
            pred = mrsc.predict()
            pred = pred[pred.index.isin(pred_metrics)]
            true = mrsc.getTrue()
            pred.columns = [playerName+" "+ str(a) for a in range(pred_interval)]
            true.columns = [playerName+" "+ str(a) for a in range(pred_interval)]
            player_pred = pd.concat([player_pred, pred], axis=0)
            player_true = pd.concat([player_true, true], axis=0)

            # plot
            for i in range(mrsc.num_k):
                metric_plt = mrsc.metrics[i]
                true_traj = mrsc.target.data[metric_plt].dropna(axis='columns').values.flatten()
                pred_traj = np.dot(mrsc.model.donor_pre.iloc[:, i*mrsc.model.interv_index:(i+1)*mrsc.model.interv_index].T, 
                                   mrsc.model.beta).flatten()

                if mrsc.weighting != None:
                    mean_pre = utils.get_preint_data(combinedDF=mrsc.weights[0].to_frame().T,
                                                    intervIndex=mrsc.interv_index, totalIndex=mrsc.total_index,
                                                    nbrMetrics=mrsc.num_k, reindex=True).values
                    var_pre = utils.get_preint_data(combinedDF=mrsc.weights[1].to_frame().T,
                                                    intervIndex=mrsc.interv_index, totalIndex=mrsc.total_index,
                                                    nbrMetrics=mrsc.num_k, reindex=True).values
                    mean_pre = mean_pre[:, i*mrsc.model.interv_index:(i+1)*mrsc.model.interv_index].flatten()
                    var_pre = var_pre[:, i*mrsc.model.interv_index:(i+1)*mrsc.model.interv_index].flatten()
                    pred_traj = (pred_traj * np.sqrt(var_pre.T)) + mean_pre.T

                markers_on = np.arange(pred_traj.shape[0], true_traj.shape[0]).tolist()
                pred_traj = np.append(pred_traj, pred[pred.index==metric_plt].values.flatten())
                #markers_on = [true_traj.shape[0]-mrsc.pred_interval]
                plot_pred(pred_traj, true_traj, markers_on, metric_plt, playerName, dir_name_player)
    
        all_pred = pd.concat([all_pred, player_pred], axis=1)
        all_true = pd.concat([all_true, player_true], axis=1)
        all_bench = pd.concat([all_bench, benchmark], axis=1)

        R2 = getR2(player_true, player_pred, benchmark)
        all_R2 = pd.concat([all_R2, R2], axis=1)

    ###################
    print("Number of metrics: {}".format(all_pred.shape[0]))
    print("Number of players: {}".format(all_pred.shape[1]))
    print()
    mask = (all_true !=0 )
    mape = np.abs(all_pred - all_true) / all_true[mask]
    print("*** MAPE ***")
    print(mape.mean(axis=1))
    print("MAPE for all: ", mape.mean().mean())

    rmse = utils.rmse_2d(all_true, all_pred)
    print()
    print("*** RMSE ***")
    print(rmse)
    print("RMSE for all: ", rmse.mean())

    print()
    print("*** R2 ***")
    print(all_R2.mean(axis=1))
    print("R2 for all: ", all_R2.mean(axis=1).mean(axis=0))

    edited_R2 = copy.deepcopy(all_R2)
    edited_R2[edited_R2 <0] = 0
    print()
    print("*** edited R2 ***")
    print(edited_R2.mean(axis=1))
    print("R2 for all: ", edited_R2.mean().mean())
    ##############################################################

In [3]:
def edit_year_count(df):
    for player in df.Player.unique():
        df.loc[df.Player == player, 'year_count'] = np.arange(len(df[df.Player == player]))
    return df

def fix_duplicates(stats, duplicate_names):
    for player in duplicate_names:
        y_true = stats[stats.Player==player]['Year'].values
        y_old = np.arange(np.min(y_true), np.min(y_true) + len(y_true))
        idx = np.nonzero(y_true - y_old)[0]
        years = y_true[idx]
        stats.loc[(stats['Player'] == player) & (stats['Year'].isin(years)), 'Player'] = player + ' Jr'
    return stats

""" 
retrive players who played in 'pred_year' who satisfy the following:
1. played at least 'min_games' in 'pred_year' 
2. played at least 'buffer' seasons, where each admissible season has > 'min_games'
"""
def get_active_players(stats_target, pred_year, buffer, min_games):
    df = stats_target.copy()    
    
    # minimum number of years a player must have played in NBA
    year_min = pred_year - buffer

    # only consider players that played at least 'min_games' in 'pred_year' 
    active = df.loc[(df.Year == pred_year) & (df.G >= min_games), 'Player'].tolist()

    # consider remaining players who played >= 'buffer' seasons (prior to 'pred_year') with > 'min_games'
    active_final = []
    df = df[df.Year < pred_year]
    
    for player in active: 
        if len(df[df.Player==player]) >= buffer:
            active_final.append(player)
    
    return active_final

# *** DATA PREPARATION ***

In [13]:
""" USER PARAMETERS """
starting_year = 1970
#min_games_donor = 40
#min_games_target = 40
#min_games = np.min((min_games_donor, min_games_target))
min_games= 30
train_year = 2015
pred_interval = 1
pred_year = train_year + pred_interval
buffer = 4
num_top_players = 300

parameters = {'starting_year': starting_year,
             #'min_games_donor': min_games_donor,
             #'min_games_target': min_games_target,
             'min_games': min_games,
             'pred_year': pred_year,
             'pred_interval': pred_interval,
             'min_num_years_played': buffer}

In [14]:
""" players dataframe """
players = pd.read_csv("../data/nba-players-stats/player_data.csv")

# sort players by (name, year_start)
players = players.sort_values(by=['name', 'year_start'])

# filter players by years considered
players = players[players.year_start >= starting_year] 

In [15]:
""" stats dataframe """
stats = pd.read_csv("../data/nba-players-stats/Seasons_Stats.csv")

# fix the name* issue
stats = stats.replace('\*','',regex=True)

# sort players by (name, year)
stats = stats.sort_values(by=['Player', 'Year'])

# remove multiple rows for the same [Year, Player]
totals = stats[stats.Tm == "TOT"]
duplicates_removed = stats.drop_duplicates(subset=["Year", "Player"], keep=False)
stats = pd.concat([duplicates_removed, totals], axis=0).sort_values("Unnamed: 0")

# filter players by years considered
stats = stats[stats.Year >= starting_year]

In [16]:
""" players + stats dataframes """
valid_players = list(set(stats.Player) & set(players.name))
stats = stats[stats['Player'].isin(valid_players)]
players = players[players['name'].isin(valid_players)]

# correct names in "players" dataframe
duplicate_names = []
for name in players.name:
    numrows = len(players[players['name']==name])
    if numrows == 2:
        duplicate_names.append(name)
        i = 0
        for birth_date in players.loc[players['name']==name, 'birth_date']:
            if i == 1:
                players.loc[(players['name']==name) & (players['birth_date']==birth_date) , 'name'] = name + ' Jr'
            i += 1
    elif numrows == 3:
        players = players[players.name != name]
         
# correct names in "stats" dataframe
stats = fix_duplicates(stats, duplicate_names)

# merge
players = players.rename(columns={"name": "Player"})
stats = pd.merge(stats, players, on='Player', how='left')
# sanity check 
stats = stats[(stats.Year >= stats.year_start) & (stats.Year <= stats.year_end)]

stats.Year = stats.Year.astype(int)
stats.year_start = stats.year_start.astype(int)
stats['year_count'] = stats.Year - stats.year_start

In [17]:
""" donor setup """
# only consider years prior to 'pred_year'
stats_donor = stats[stats.Year < pred_year]
stats_donor = stats_donor.sort_values(by=['Player', 'Year'])

# only consider years in which at least "min_games" number of games were played
stats_donor = stats_donor[stats_donor.G >= min_games]

# edit 'year_count'
#stats_donor = edit_year_count(stats_donor)

# create donor object
allPivotedTableDict_d, allMetrics = prepareData(stats_donor)
donor = Donor(allPivotedTableDict_d)

In [18]:
""" target setup """
# consider all years up to (and including) 'pred_year'
stats_target = stats[stats.Year <= pred_year]

# exclude years prior to 'pred_year' in which < 'min_games' were played
idx = stats_target.loc[(stats_target.G < min_games) & (stats_target.Year < pred_year)].index
stats_target.drop(idx, inplace=True)

# edit 'year_count'
#stats_target = edit_year_count(stats_target)

# create target dictionary of values
allPivotedTableDict, allMetrics = prepareData(stats_target)

# get target player names
targetNames = get_active_players(stats_target, pred_year, buffer, min_games) 
targetNames.sort()

if 'Kevin Garnett' in targetNames: 
    targetNames.remove("Kevin Garnett")
if 'Kobe Bryant' in targetNames:
    targetNames.remove("Kobe Bryant")

print("*** DONE! ***")

*** DONE! ***


In [19]:
stats_target[stats_target.Player=='Al Horford']

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,PF,PTS,year_start,year_end,position,height,weight,birth_date,college,year_count
12508,18989,2008,Al Horford,C,21.0,ATL,81.0,77.0,2540.0,14.7,...,267.0,821.0,2008,2018.0,C-F,6-10,245.0,"June 3, 1986",University of Florida,0
12956,19586,2009,Al Horford,C,22.0,ATL,67.0,67.0,2242.0,17.0,...,190.0,773.0,2008,2018.0,C-F,6-10,245.0,"June 3, 1986",University of Florida,1
13407,20163,2010,Al Horford,C,23.0,ATL,81.0,81.0,2845.0,19.4,...,225.0,1148.0,2008,2018.0,C-F,6-10,245.0,"June 3, 1986",University of Florida,2
13857,20793,2011,Al Horford,C,24.0,ATL,77.0,77.0,2704.0,20.7,...,193.0,1178.0,2008,2018.0,C-F,6-10,245.0,"June 3, 1986",University of Florida,3
14770,21918,2013,Al Horford,C,26.0,ATL,74.0,74.0,2756.0,19.8,...,163.0,1289.0,2008,2018.0,C-F,6-10,245.0,"June 3, 1986",University of Florida,5
15737,23139,2015,Al Horford,C,28.0,ATL,76.0,76.0,2318.0,21.4,...,121.0,1156.0,2008,2018.0,C-F,6-10,245.0,"June 3, 1986",University of Florida,7
16212,23762,2016,Al Horford,C,29.0,ATL,82.0,82.0,2631.0,19.4,...,163.0,1249.0,2008,2018.0,C-F,6-10,245.0,"June 3, 1986",University of Florida,8


In [20]:
allPivotedTableDict['PTS_G']

year_count,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
Player,,,,,,,,,,,,,,,,,,,,,
A.C. Green,6.353659,10.784810,11.426829,13.268293,12.939024,9.146341,13.609756,12.817073,14.682927,11.170732,...,7.192771,7.317073,4.920000,5.036585,4.475610,NaN,NaN,NaN,NaN,NaN
A.J. English,8.800000,10.938272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A.J. Guyton,6.000000,5.422222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A.J. Price,7.321429,6.460000,3.909091,7.736842,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aaron Brooks,5.176471,11.175000,19.560976,10.745763,NaN,7.094340,8.958333,11.634146,7.115942,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aaron Gordon,5.170213,9.217949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aaron Gray,4.295082,3.500000,3.250000,3.146341,3.897959,2.833333,1.756757,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aaron Harrison,0.857143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aaron James,11.671053,9.026667,10.865385,12.162500,9.958904,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# *** EXPERIMENTAL SECTION ***

In [37]:
predMetrics = ["PTS_G","AST_G","TOV_G","FG%","FT%","3P_G","TRB_G","STL_G","BLK_G"]
offMetrics = ["PTS_G","AST_G","TOV_G","PER_w", "FG%","FT%","3P_G"]
defMetrics = ["TRB_G","STL_G","BLK_G"]

In [38]:
"""
setup
"""
# user input
donor_window_type = 'sliding'
normalize_metric = None 
threshold = 0.98
helper_metrics = []
num_top = 50
top_players = getTopPlayers(stats, pred_year+pred_interval, 'PTS', num_top).values.flatten().tolist()

In [39]:
""" 
directory path
"""
# donor window type
donor_window_label = donor_window_type + '/'

# prediction method
helper_metrics_label = ''
if helper_metrics:
    pred_method = 'mrsc/'
    for helper_metric in helper_metrics: 
        helper_metrics_label = helper_metrics_label + helper_metric + '_'
    helper_metrics_label += '/'
else:
    pred_method = 'rsc/'

# metric normalizing type
if normalize_metric == None:
    normalize_metric_label = 'no_normalization/'
else:
    normalize_metric_label = normalize_metric
    
# singular value threshold energy level 
threshold_label = str(threshold*100)[:2] + '/'

# prediction length
pred_length_label = str(pred_interval) + 'year/'
    
dir_name = 'plots/' + pred_method + donor_window_label + normalize_metric_label + helper_metrics_label + pred_length_label
    
print(dir_name)

plots/rsc/sliding/no_normalization/1year/


In [40]:
# setup 
donorSetup= [normalize_metric, donor_window_type, True]
denoiseSetup = ["SVD", "all"]
regression_method = "pinv"
threshold = 0.98
verbose = False
metrics_list = [[metric] + helper_metrics for metric in predMetrics]
print(metrics_list)

[['PTS_G'], ['AST_G'], ['TOV_G'], ['FG%'], ['FT%'], ['3P_G'], ['TRB_G'], ['STL_G'], ['BLK_G']]


In [41]:
"""
computation
"""
print("Computing...")
print()

print("*** SETUP ***")
for key, value in parameters.items():
    print("{}: {}".format(key, value))
print()
print("Experiment: {}".format(dir_name))
print() 

annual_predictions(targetNames, allPivotedTableDict, donor, pred_interval, metrics_list, predMetrics,
                   threshold, donorSetup, denoiseSetup, regression_method, verbose, dir_name, top_players) 

Computing...

*** SETUP ***
starting_year: 1970
min_games: 30
pred_year: 2015
pred_interval: 1
min_num_years_played: 4

Experiment: plots/rsc/sliding/no_normalization/1year/

Number of metrics: 9
Number of players: 168

*** MAPE ***
PTS_G    0.270502
AST_G    0.340965
TOV_G    0.326014
FG%      0.069281
FT%      0.064955
3P_G     0.984235
TRB_G    0.262016
STL_G    0.273317
BLK_G    0.428644
dtype: float64
MAPE for all:  0.32189798602333836

*** RMSE ***
PTS_G    2.831605
AST_G    1.025143
TOV_G    0.464593
FG%      0.040973
FT%      0.083110
3P_G     0.826983
TRB_G    1.377779
STL_G    0.249559
BLK_G    0.203486
dtype: float64
RMSE for all:  0.7892478980858849

*** R2 ***
PTS_G   -3.549194e+01
AST_G   -1.623508e+05
TOV_G   -1.999485e+02
FG%     -5.989944e+07
FT%     -1.489740e+00
3P_G    -4.127368e+02
TRB_G   -1.078143e+01
STL_G   -9.479724e+01
BLK_G   -1.293423e+01
dtype: float64
R2 for all:  -6673617.816801179

*** edited R2 ***
PTS_G    0.467140
AST_G    0.429289
TOV_G    0.417246


In [ ]:
Computing...

*** SETUP ***
starting_year: 1970
min_games: 30
pred_year: 2015
pred_interval: 1
min_num_years_played: 4

Experiment: plots/rsc/fixed/no_normalization/1year/

Number of metrics: 9
Number of players: 168

*** MAPE ***
PTS_G    0.281630
AST_G    0.383883
TOV_G    0.367258
FG%      0.066654
FT%      0.064001
3P_G     2.231075
TRB_G    0.240714
STL_G    0.289260
BLK_G    0.812528
dtype: float64
MAPE for all:  0.4890711028769686

*** RMSE ***
PTS_G    2.870007
AST_G    1.130413
TOV_G    0.521150
FG%      0.039212
FT%      0.083259
3P_G     2.051136
TRB_G    1.234394
STL_G    0.275070
BLK_G    0.632836
dtype: float64
RMSE for all:  0.981942017887479

*** R2 ***
PTS_G   -9.652059e+01
AST_G   -1.401625e+05
TOV_G   -2.403465e+02
FG%     -4.108149e+07
FT%     -2.043987e+02
3P_G    -1.104356e+04
TRB_G   -1.557059e+01
STL_G   -1.400594e+02
BLK_G   -2.228222e+02
dtype: float64
R2 for all:  -4581512.717173296

*** edited R2 ***
PTS_G    0.480122
AST_G    0.399452
TOV_G    0.392016
FG%      0.258035
FT%      0.272377
3P_G     0.184772
TRB_G    0.425264
STL_G    0.371070
BLK_G    0.349180
dtype: float64
R2 for all:  0.34906402335974507

In [13]:
*** SETUP ***
starting_year: 1970
min_games: 30
pred_year: 2015
pred_interval: 1
min_num_years_played: 4

Experiment: plots/rsc/sliding/no_normalization/1year/

Number of metrics: 9
Number of players: 229

*** MAPE ***
PTS_G    0.309890
AST_G    0.393498
TOV_G    0.334534
FG%      0.071718
FT%      0.070794
3P_G     0.956313
TRB_G    0.288091
STL_G    0.343604
BLK_G    0.475623
dtype: float64
MAPE for all:  0.3472343057384938

*** RMSE ***
PTS_G    2.864969
AST_G    0.985771
TOV_G    0.447866
FG%      0.041900
FT%      0.102549
3P_G     0.826434
TRB_G    1.411899
STL_G    0.263138
BLK_G    0.229166
dtype: float64
RMSE for all:  0.7970768413822888

*** R2 ***
PTS_G   -6.298931e+01
AST_G   -2.511737e+02
TOV_G   -1.536598e+02
FG%     -4.394371e+07
FT%     -9.724013e-01
3P_G    -2.814039e+02
TRB_G   -1.934948e+02
STL_G   -1.101312e+04
BLK_G   -2.092002e+01
dtype: float64
R2 for all:  -4883965.652321282

*** edited R2 ***
PTS_G    0.460665
AST_G    0.413033
TOV_G    0.426915
FG%      0.280209
FT%      0.081732
3P_G     0.267270
TRB_G    0.378749
STL_G    0.326196
BLK_G    0.357458
dtype: float64
R2 for all:  0.33297035707737216

In [14]:
target.key

'Al Jefferson'

# DEBUGGING STAGE

In [42]:
playerName = 'Al Jefferson'
target = Target(playerName, allPivotedTableDict)
mrsc = mRSC(donor, target, pred_interval, probObservation=1)        
player_pred = pd.DataFrame()
player_true = pd.DataFrame()
        
"""# benchmark
true, benchmark = getBenchmark(target, predMetrics, pred_interval)
benchmark"""

'# benchmark\ntrue, benchmark = getBenchmark(target, predMetrics, pred_interval)\nbenchmark'

In [44]:
metric = metrics_list[0]

metric = predMetrics
mrsc.fit_threshold(metric, threshold, donorSetup, denoiseSetup,regression_method, verbose)

TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

In [59]:
metric = metrics_list[0]

metric = predMetrics
mrsc.fit_threshold(metric, threshold, donorSetup, denoiseSetup,regression_method, verbose)
pred = mrsc.predict()
pred = pred[pred.index.isin(predMetrics)]
true = mrsc.getTrue()
pred.columns = [playerName+" "+ str(a) for a in range(pred_interval)]
true.columns = [playerName+" "+ str(a) for a in range(pred_interval)]
player_pred = pd.concat([player_pred, pred], axis=0)
player_true = pd.concat([player_true, true], axis=0)

In [69]:
for i in range(mrsc.num_k):
    metric_plt = mrsc.metrics[i]
    print(pred[pred.index == metric_plt].values.flatten())

[14.385505363683219 12.911161072448229]
[1.702742145919105 1.5253697807658821]
[1.7192385770545187 1.564441901184189]
[0.48319136019142295 0.48629219665166257]
[0.7249183532344486 0.7164523271716283]
[0.3539001138955636 0.353363752435545]
[7.309900620690225 6.731121626472695]
[0.7971118479166359 0.7140108132435534]
[0.9611268100242883 0.8402310462167943]


In [61]:
pred_list = pred.values.flatten()
pred_list

array([14.385505363683219, 12.911161072448229, 1.702742145919105,
       1.5253697807658821, 1.7192385770545187, 1.564441901184189,
       0.48319136019142295, 0.48629219665166257, 0.7249183532344486,
       0.7164523271716283, 0.3539001138955636, 0.353363752435545,
       7.309900620690225, 6.731121626472695, 0.7971118479166359,
       0.7140108132435534, 0.9611268100242883, 0.8402310462167943],
      dtype=object)

In [53]:
i = 0
metric_plt = mrsc.metrics[i]
true_traj = mrsc.target.data[metric_plt].dropna(axis='columns').values.flatten()
true_traj.shape

(13,)

In [56]:
pred_traj = np.dot(mrsc.model.donor_pre.iloc[:, i*mrsc.model.interv_index:(i+1)*mrsc.model.interv_index].T, 
                                   mrsc.model.beta).flatten()
pred_traj = np.append(pred_traj, pred_list[i])

NameError: name 'pred_list' is not defined

In [ ]:
"""
targets
"""
activePlayers = getActivePlayers(stats, pred_year+pred_interval, buffer=buffer)
topPlayers = getTopPlayers(stats, pred_year, 'PTS', num_top_players).values.flatten().tolist()
#topPlayers = topPlayers.values.flatten().tolist()

print("non-rookie active players in 2016: ", len(activePlayers))

# first group
print("***** First Group *****")
target_names_1 = list(set(activePlayers) & set(topPlayers))
max_total_index = 20
print("* not sufficient donor pool:")
for playerName in target_names_1:
    target = Target(playerName, allPivotedTableDict)
    if (target.total_index > max_total_index):
        target_names_1.remove(playerName)
        print(playerName)
print("* total number of target players: ", len(target_names_1))

# second group
print()
print("***** Second Group *****")
target_names_2 = activePlayers
max_total_index = 20
print("* not sufficient donor pool:")
for playerName in target_names_2:
    target = Target(playerName, allPivotedTableDict)
    if (target.total_index > max_total_index):
        target_names_2.remove(playerName)
        print(playerName)
print("* total number of target players: ", len(target_names_2))
print()

##############
# final
targetNames = target_names_2
targetNames.sort()

if 'Kevin Garnett' in targetNames: 
    targetNames.remove("Kevin Garnett")
if 'Kobe Bryant' in targetNames:
    targetNames.remove("Kobe Bryant")

print("*** DONE! ***")

In [ ]:
metrics_to_use = allMetrics

print("Algo: outputs the mean of the player's history")
print("-----")
pred_all = pd.DataFrame()
true_all = pd.DataFrame()
for playerName in targetNames:
    target = Target(playerName, allPivotedTableDict)
    true, pred = getBenchmark(target, metrics_to_use, pred_interval)
    
    pred_all = pd.concat([pred_all, pred], axis=1)
    true_all = pd.concat([true_all, true], axis=1)

###################
mask = (true_all !=0 )
mape = np.abs(pred_all - true_all) / true_all[mask]
print("*** MAPE ***")
print(mape.mean(axis=1))
print("MAPE for all: ", mape.mean().mean()) 

In [295]:
"""
experiment setup
"""
# overall setup
donorSetup= [None,"fixed", True]
denoiseSetup = ["SVD", "all"]
regression_method = "pinv"

threshold = 0.97
verbose = False

metrics_list = [[x] for x in allMetrics]

"""
experiment
"""
print("1 metric at once")

all_pred = pd.DataFrame()
all_true = pd.DataFrame()
all_bench = pd.DataFrame()
all_R2 = pd.DataFrame()
for playerName in playerNames:
    target = Target(playerName, allPivotedTableDict)
    
    # benchmark
    true, benchmark = getBenchmark(target, ["PTS_G"], pred_interval)
    
    # prediction
    mrsc = mRSC(donor, target, pred_interval, probObservation=1)
    player_pred = pd.DataFrame()
    player_true = pd.DataFrame()
    for i in range(len(metrics_list)):
        mrsc.fit_threshold(metrics_list[i], threshold, donorSetup, denoiseSetup,regression_method, verbose)
        pred = mrsc.predict()
        true = mrsc.getTrue()
        pred.columns = [playerName+" "+ str(a) for a in range(pred_interval)]
        true.columns = [playerName+" "+ str(a) for a in range(pred_interval)]
        player_pred = pd.concat([player_pred, pred], axis=0)
        player_true = pd.concat([player_true, true], axis=0)
    all_pred = pd.concat([all_pred, player_pred], axis=1)
    all_true = pd.concat([all_true, player_true], axis=1)
    all_bench = pd.concat([all_bench, benchmark], axis=1)
    
    R2 = getR2(player_true, player_pred, benchmark)
    all_R2 = pd.concat([all_R2, R2], axis=1)

##################
print(all_pred.shape)
mask = (all_true !=0 )
mape = np.abs(all_pred - all_true) / all_true[mask]
print("*** MAPE ***")
print(mape.mean(axis=1))
print("MAPE for all: ", mape.mean().mean())

rmse = utils.rmse_2d(all_true, all_pred)
print()
print("*** RMSE ***")
print(rmse)
print("RMSE for all: ", rmse.mean())

print()
print("*** R2 ***")
print(all_R2.mean(axis=1))
print("R2 for all: ", all_R2.mean(axis=1).mean(axis=0))

edited_R2 = copy.deepcopy(all_R2)
edited_R2[edited_R2 <0] = 0
print()
print("*** edited R2 ***")
print(edited_R2.mean(axis=1))
print("R2 for all: ", edited_R2.mean().mean())
##############################################################

1 metric at once
(10, 134)
*** MAPE ***
PTS_G    0.277280
AST_G    0.303132
TOV_G    0.281037
TRB_G    0.232480
STL_G    0.268172
BLK_G    0.371174
3P_G     0.897202
FG%      0.060781
FT%      0.061157
PER_w    0.174422
dtype: float64
MAPE for all:  0.2810383781032388

*** RMSE ***
PTS_G    3.078215
AST_G    0.939275
TOV_G    0.421465
TRB_G    1.394864
STL_G    0.235441
BLK_G    0.195664
3P_G     0.874437
FG%      0.035825
FT%      0.058761
PER_w    2.891542
dtype: float64
RMSE for all:  1.012548902251213

*** R2 ***
PTS_G   -68.137314
AST_G     0.989566
TOV_G     0.997875
TRB_G    -4.560719
STL_G     0.999339
BLK_G     0.999462
3P_G      0.990676
FG%       0.999979
FT%       0.999935
PER_w   -14.244623
dtype: float64
R2 for all:  -7.996582330214866

*** edited R2 ***
PTS_G    0.423362
AST_G    0.989566
TOV_G    0.997875
TRB_G    0.856446
STL_G    0.999339
BLK_G    0.999462
3P_G     0.990676
FG%      0.999979
FT%      0.999935
PER_w    0.603814
dtype: float64
R2 for all:  0.88604549089

In [294]:
"""
experiment setup
"""
# overall setup
donorSetup= [None,"sliding", True]
denoiseSetup = ["SVD", "all"]
regression_method = "pinv"

threshold = 0.97
verbose = False

metrics_list = [[x] for x in allMetrics]

"""
experiment
"""
print("1 metric at once")

all_pred = pd.DataFrame()
all_true = pd.DataFrame()
all_bench = pd.DataFrame()
all_R2 = pd.DataFrame()
for playerName in playerNames:
    target = Target(playerName, allPivotedTableDict)
    
    # benchmark
    true, benchmark = getBenchmark(target, ["PTS_G"], pred_interval)
    
    # prediction
    mrsc = mRSC(donor, target, pred_interval, probObservation=1)
    player_pred = pd.DataFrame()
    player_true = pd.DataFrame()
    for i in range(len(metrics_list)):
        mrsc.fit_threshold(metrics_list[i], threshold, donorSetup, denoiseSetup,regression_method, verbose)
        pred = mrsc.predict()
        true = mrsc.getTrue()
        pred.columns = [playerName+" "+ str(a) for a in range(pred_interval)]
        true.columns = [playerName+" "+ str(a) for a in range(pred_interval)]
        player_pred = pd.concat([player_pred, pred], axis=0)
        player_true = pd.concat([player_true, true], axis=0)
    all_pred = pd.concat([all_pred, player_pred], axis=1)
    all_true = pd.concat([all_true, player_true], axis=1)
    all_bench = pd.concat([all_bench, benchmark], axis=1)
    
    R2 = getR2(player_true, player_pred, benchmark)
    all_R2 = pd.concat([all_R2, R2], axis=1)

##################
print(all_pred.shape)
mask = (all_true !=0 )
mape = np.abs(all_pred - all_true) / all_true[mask]
print("*** MAPE ***")
print(mape.mean(axis=1))
print("MAPE for all: ", mape.mean().mean())

rmse = utils.rmse_2d(all_true, all_pred)
print()
print("*** RMSE ***")
print(rmse)
print("RMSE for all: ", rmse.mean())

print()
print("*** R2 ***")
print(all_R2.mean(axis=1))
print("R2 for all: ", all_R2.mean(axis=1).mean(axis=0))

edited_R2 = copy.deepcopy(all_R2)
edited_R2[edited_R2 <0] = 0
print()
print("*** edited R2 ***")
print(edited_R2.mean(axis=1))
print("R2 for all: ", edited_R2.mean().mean())
##############################################################

1 metric at once
(10, 134)
*** MAPE ***
PTS_G    0.285066
AST_G    0.296102
TOV_G    0.277888
TRB_G    0.221688
STL_G    0.271170
BLK_G    0.378055
3P_G     0.620813
FG%      0.062568
FT%      0.061464
PER_w    0.176787
dtype: float64
MAPE for all:  0.2583693415639397

*** RMSE ***
PTS_G    3.125182
AST_G    0.935674
TOV_G    0.420983
TRB_G    1.367314
STL_G    0.240406
BLK_G    0.204238
3P_G     0.490811
FG%      0.037340
FT%      0.058443
PER_w    3.152154
dtype: float64
RMSE for all:  1.003254594593438

*** R2 ***
PTS_G   -88.662326
AST_G     0.988888
TOV_G     0.997892
TRB_G    -1.950227
STL_G     0.999319
BLK_G     0.999350
3P_G      0.997055
FG%       0.999977
FT%       0.999936
PER_w   -10.868037
dtype: float64
R2 for all:  -9.449817350209434

*** edited R2 ***
PTS_G    0.411329
AST_G    0.988888
TOV_G    0.997892
TRB_G    0.836763
STL_G    0.999319
BLK_G    0.999350
3P_G     0.997055
FG%      0.999977
FT%      0.999936
PER_w    0.569363
dtype: float64
R2 for all:  0.87998723292

# Grouping

In [269]:
# check if grouping with n=2 would work
"""
experiment setup
"""
# overall setup
donorSetup= [None,"Sliding", True]
denoiseSetup = ["SVD", "all"]
regression_method = "pinv"

threshold = 0.97
verbose = False

"""
experiment
"""
result = pd.DataFrame(index=allMetrics, columns=allMetrics)
for m1 in range(len(allMetrics)):
    for m2 in range(len(allMetrics)):
        metric1 = allMetrics[m1]
        metric2 = allMetrics[m2]
        metrics = [metric1, metric2]
        metrics_list = [metrics]
        print()
        print("***************************************")
        print(metrics)

        all_pred = pd.DataFrame()
        all_true = pd.DataFrame()
        all_bench = pd.DataFrame()
        all_R2 = pd.DataFrame()
        for playerName in playerNames:
            target = Target(playerName, allPivotedTableDict)

            # benchmark
            true, benchmark = getBenchmark(target, metrics, pred_interval)

            # prediction
            mrsc = mRSC(donor, target, pred_interval, probObservation=1)
            player_pred = pd.DataFrame()
            player_true = pd.DataFrame()
            for i in range(len(metrics_list)):
                mrsc.fit_threshold(metrics_list[i], threshold, donorSetup, denoiseSetup,regression_method, verbose)
                pred = mrsc.predict()
                true = mrsc.getTrue()
                pred.columns = [playerName+" "+ str(a) for a in range(pred_interval)]
                true.columns = [playerName+" "+ str(a) for a in range(pred_interval)]
                player_pred = pd.concat([player_pred, pred], axis=0)
                player_true = pd.concat([player_true, true], axis=0)
            all_pred = pd.concat([all_pred, player_pred], axis=1)
            all_true = pd.concat([all_true, player_true], axis=1)
            all_bench = pd.concat([all_bench, benchmark], axis=1)

            R2 = getR2(player_true, player_pred, benchmark)
            all_R2 = pd.concat([all_R2, R2], axis=1)

        ##################
#         print(all_pred.shape)
        mask = (all_true !=0 )
        mape = np.abs(all_pred - all_true) / all_true[mask]
        print("*** MAPE ***")
        print(mape.mean(axis=1))
        result.iloc[m1,m2] = mape.mean(axis=1).values[0]
print(result)
diff = (result.T - mape.mean(axis=1).values).T


***************************************
['PTS_G', 'PTS_G']
*** MAPE ***
PTS_G    0.285066
PTS_G    0.285066
dtype: float64

***************************************
['PTS_G', 'AST_G']
*** MAPE ***
PTS_G    0.283337
AST_G    0.409922
dtype: float64

***************************************
['PTS_G', 'TOV_G']
*** MAPE ***
PTS_G    0.285706
TOV_G    0.393103
dtype: float64

***************************************
['PTS_G', 'TRB_G']
*** MAPE ***
PTS_G    0.300432
TRB_G    0.244106
dtype: float64

***************************************
['PTS_G', 'STL_G']
*** MAPE ***
PTS_G    0.284957
STL_G    0.421654
dtype: float64

***************************************
['PTS_G', 'BLK_G']
*** MAPE ***
PTS_G    0.285237
BLK_G    1.371677
dtype: float64

***************************************
['PTS_G', '3P_G']
*** MAPE ***
PTS_G    0.285382
3P_G     2.160642
dtype: float64

***************************************
['PTS_G', 'FG%']
*** MAPE ***
PTS_G    0.285094
FG%      0.337157
dtype: float64

**********

*** MAPE ***
3P_G    0.620813
3P_G    0.620813
dtype: float64

***************************************
['3P_G', 'FG%']
*** MAPE ***
3P_G    0.609159
FG%     0.063301
dtype: float64

***************************************
['3P_G', 'FT%']
*** MAPE ***
3P_G    0.519625
FT%     0.061411
dtype: float64

***************************************
['3P_G', 'PER_w']
*** MAPE ***
3P_G     2.444566
PER_w    0.177107
dtype: float64

***************************************
['FG%', 'PTS_G']
*** MAPE ***
FG%      0.337157
PTS_G    0.285094
dtype: float64

***************************************
['FG%', 'AST_G']
*** MAPE ***
FG%      0.540236
AST_G    0.300862
dtype: float64

***************************************
['FG%', 'TOV_G']
*** MAPE ***
FG%      0.347302
TOV_G    0.280622
dtype: float64

***************************************
['FG%', 'TRB_G']
*** MAPE ***
FG%      0.362217
TRB_G    0.230544
dtype: float64

***************************************
['FG%', 'STL_G']
*** MAPE ***
FG%      0.062644


In [290]:
diff = (result.T - mape.mean(axis=1).values).T
diff[diff<0].astype(float).round(3)

,PTS_G,AST_G,TOV_G,TRB_G,STL_G,BLK_G,3P_G,FG%,FT%,PER_w
PTS_G,NaN,-0.002,NaN,NaN,-0.000,NaN,NaN,NaN,NaN,NaN
AST_G,NaN,NaN,NaN,NaN,NaN,NaN,-0.003,NaN,-0.005,NaN
TOV_G,NaN,-0.017,NaN,NaN,NaN,NaN,-0.018,NaN,-0.012,NaN
TRB_G,NaN,NaN,NaN,-0.0,NaN,NaN,NaN,NaN,NaN,NaN
STL_G,NaN,-0.003,NaN,NaN,-0.000,-0.009,NaN,NaN,-0.001,NaN
BLK_G,NaN,NaN,NaN,NaN,-0.006,NaN,NaN,-0.003,-0.004,NaN
3P_G,NaN,NaN,NaN,NaN,NaN,-0.018,-0.000,-0.012,-0.101,NaN
FG%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FT%,NaN,NaN,NaN,NaN,NaN,-0.001,-0.000,NaN,NaN,NaN
PER_w,NaN,NaN,NaN,NaN,NaN,-0.000,NaN,-0.000,NaN,NaN


# Weighting

In [ ]:
# metrics_list = [['PTS_G'],
#  ['AST_G'],
#  ['TOV_G'],
#  ['TRB_G'],
#  ['STL_G'],
#  ['BLK_G'],
#  ['3P_G'],
#  ['FG%'],
#  ['FT%','3P_G'],
#  ['PER_w']]

metrics_list = [[x] for x in allMetrics]

In [324]:
"""
experiment setup
"""
print("variance")
# overall setup
donorSetup= ["variance","sliding", True]
denoiseSetup = ["SVD", "all"]
regression_method = "pinv"

threshold = 0.97
verbose = False

"""
experiment
"""
all_pred = pd.DataFrame()
all_true = pd.DataFrame()
all_bench = pd.DataFrame()
all_R2 = pd.DataFrame()
for playerName in playerNames:
    target = Target(playerName, allPivotedTableDict)
    
    # benchmark
    true, benchmark = getBenchmark(target, ["PTS_G"], pred_interval)
    
    # prediction
    mrsc = mRSC(donor, target, pred_interval, probObservation=1)
    player_pred = pd.DataFrame()
    player_true = pd.DataFrame()
    for i in range(len(metrics_list)):
        mrsc.fit_threshold(metrics_list[i], threshold, donorSetup, denoiseSetup,regression_method, verbose)
        pred = mrsc.predict()
        true = mrsc.getTrue()
        pred.columns = [playerName+" "+ str(a) for a in range(pred_interval)]
        true.columns = [playerName+" "+ str(a) for a in range(pred_interval)]
        player_pred = pd.concat([player_pred, pred], axis=0)
        player_true = pd.concat([player_true, true], axis=0)
    all_pred = pd.concat([all_pred, player_pred], axis=1)
    all_true = pd.concat([all_true, player_true], axis=1)
    all_bench = pd.concat([all_bench, benchmark], axis=1)
    
    R2 = getR2(player_true, player_pred, benchmark)
    all_R2 = pd.concat([all_R2, R2], axis=1)

##################
print(all_pred.shape)
mask = (all_true !=0 )
mape = np.abs(all_pred - all_true) / all_true[mask]
print("*** MAPE ***")
print(mape.mean(axis=1))
print("MAPE for all: ", mape.mean().mean())

rmse = utils.rmse_2d(all_true, all_pred)
print()
print("*** RMSE ***")
print(rmse)
print("RMSE for all: ", rmse.mean())

print()
print("*** R2 ***")
print(all_R2.mean(axis=1))
print("R2 for all: ", all_R2.mean(axis=1).mean(axis=0))

edited_R2 = copy.deepcopy(all_R2)
edited_R2[edited_R2 <0] = 0
print()
print("*** edited R2 ***")
print(edited_R2.mean(axis=1))
print("R2 for all: ", edited_R2.mean().mean())
##############################################################

variance
(10, 134)
*** MAPE ***
PTS_G    0.283308
AST_G    0.293132
TOV_G    0.276494
TRB_G    0.218923
STL_G    0.272439
BLK_G    0.425095
3P_G     0.558330
FG%      0.062566
FT%      0.062136
PER_w    0.178784
dtype: float64
MAPE for all:  0.2574142131362448

*** RMSE ***
PTS_G    3.108842
AST_G    0.935796
TOV_G    0.418640
TRB_G    1.336256
STL_G    0.234032
BLK_G    0.193097
3P_G     0.452773
FG%      0.037343
FT%      0.059003
PER_w    3.196666
dtype: float64
RMSE for all:  0.9972447301011511

*** R2 ***
PTS_G   -88.041827
AST_G     0.988982
TOV_G     0.997898
TRB_G    -1.653613
STL_G     0.999367
BLK_G     0.999376
3P_G      0.997531
FG%       0.999976
FT%       0.999935
PER_w   -11.083148
dtype: float64
R2 for all:  -9.37955221414564

*** edited R2 ***
PTS_G    0.412167
AST_G    0.988982
TOV_G    0.997898
TRB_G    0.840461
STL_G    0.999367
BLK_G    0.999376
3P_G     0.997531
FG%      0.999976
FT%      0.999935
PER_w    0.563459
dtype: float64
R2 for all:  0.8799152469536123


In [325]:
"""
experiment setup
"""
print("normalize")
# overall setup
donorSetup= ["normalize","sliding", True]
denoiseSetup = ["SVD", "all"]
regression_method = "pinv"

threshold = 0.97
verbose = False

"""
experiment
"""
all_pred = pd.DataFrame()
all_true = pd.DataFrame()
all_bench = pd.DataFrame()
all_R2 = pd.DataFrame()
for playerName in playerNames:
    target = Target(playerName, allPivotedTableDict)
    
    # benchmark
    true, benchmark = getBenchmark(target, ["PTS_G"], pred_interval)
    
    # prediction
    mrsc = mRSC(donor, target, pred_interval, probObservation=1)
    player_pred = pd.DataFrame()
    player_true = pd.DataFrame()
    for i in range(len(metrics_list)):
        mrsc.fit_threshold(metrics_list[i], threshold, donorSetup, denoiseSetup,regression_method, verbose)
        pred = mrsc.predict()
        true = mrsc.getTrue()
        pred.columns = [playerName+" "+ str(a) for a in range(pred_interval)]
        true.columns = [playerName+" "+ str(a) for a in range(pred_interval)]
        player_pred = pd.concat([player_pred, pred], axis=0)
        player_true = pd.concat([player_true, true], axis=0)
    all_pred = pd.concat([all_pred, player_pred], axis=1)
    all_true = pd.concat([all_true, player_true], axis=1)
    all_bench = pd.concat([all_bench, benchmark], axis=1)
    
    R2 = getR2(player_true, player_pred, benchmark)
    all_R2 = pd.concat([all_R2, R2], axis=1)

##################
print(all_pred.shape)
mask = (all_true !=0 )
mape = np.abs(all_pred - all_true) / all_true[mask]
print("*** MAPE ***")
print(mape.mean(axis=1))
print("MAPE for all: ", mape.mean().mean())

rmse = utils.rmse_2d(all_true, all_pred)
print()
print("*** RMSE ***")
print(rmse)
print("RMSE for all: ", rmse.mean())

print()
print("*** R2 ***")
print(all_R2.mean(axis=1))
print("R2 for all: ", all_R2.mean(axis=1).mean(axis=0))

edited_R2 = copy.deepcopy(all_R2)
edited_R2[edited_R2 <0] = 0
print()
print("*** edited R2 ***")
print(edited_R2.mean(axis=1))
print("R2 for all: ", edited_R2.mean().mean())
##############################################################

normalize
(10, 134)
*** MAPE ***
PTS_G     0.473115
AST_G     0.382205
TOV_G     0.398554
TRB_G     0.326884
STL_G     0.747976
BLK_G     0.641573
3P_G      1.034586
FG%       0.199636
FT%      11.435996
PER_w     0.539404
dtype: float64
MAPE for all:  1.6179340352100255

*** RMSE ***
PTS_G     4.930995
AST_G     1.202499
TOV_G     0.591635
TRB_G     1.817813
STL_G     0.637192
BLK_G     0.351938
3P_G      0.742832
FG%       0.133928
FT%      24.627570
PER_w    10.199659
dtype: float64
RMSE for all:  4.523606149794006

*** R2 ***
PTS_G   -229.886493
AST_G      0.977291
TOV_G      0.995292
TRB_G    -15.726024
STL_G      0.994958
BLK_G      0.997905
3P_G       0.992850
FG%        0.999770
FT%       -6.410267
PER_w    -97.628441
dtype: float64
R2 for all:  -34.36931588960094

*** edited R2 ***
PTS_G    0.302402
AST_G    0.977291
TOV_G    0.995292
TRB_G    0.834267
STL_G    0.994958
BLK_G    0.997905
3P_G     0.992850
FG%      0.999770
FT%      0.804334
PER_w    0.282870
dtype: float64
R2 